# Package installation

In [ ]:
!pip install --upgrade watson-machine-learning-client --no-cache 
!pip install --upgrade numpy --no-cache 
!pip install --upgrade lime --no-cache 
!pip install --upgrade SciPy --no-cache 

In [ ]:
WML_CREDENTIALS = {
  "apikey": "haikTOuwibItbgRYtDwuTD1Nnht8bIN96LeoSHihp4Db",
  "iam_apikey_description": "Auto generated apikey during resource-key operation for Instance - crn:v1:bluemix:public:pm-20:us-south:a/97deeb0b7e78431438a00a04f20580b7:8385ac85-436d-470e-8e67-54c492adb554::",
  "iam_apikey_name": "auto-generated-apikey-d7945c16-a7df-479f-84e8-56a1ea40416b",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/97deeb0b7e78431438a00a04f20580b7::serviceid:ServiceId-c35b5b3e-030f-4cde-9edc-9f234768fe85",
  "instance_id": "8385ac85-436d-470e-8e67-54c492adb554",
  "password": "8726f331-3271-4456-b593-e90675da02aa",
  "url": "https://us-south.ml.cloud.ibm.com",
  "username": "d7945c16-a7df-479f-84e8-56a1ea40416b"
}

## Load the training data from github

In [ ]:

import types
import pandas as pd
import numpy as np
from botocore.client import Config
import ibm_boto3

In [ ]:

import types
import pandas as pd
import numpy as np
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_a6ce39d216194fb8a502a27492c8b085 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='v2Mo9It3k1mx1nBmyYNUC3v8yDLkVkacud6QufD3ppqf',
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_a6ce39d216194fb8a502a27492c8b085.get_object(Bucket='equipmentfailuredemo-donotdelete-pr-ae2wrxuzdldzsf',Key='EQUIPMENT_FAILURE.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

pd_data = pd.read_csv(body)
pd_data.head()



In [ ]:
pd_data.head()

In [ ]:
pd_data.columns

In [ ]:
tips_summed = pd_data.groupby(['EQUIPMENT_FAILURE'])['S1'].count()
tips_summed


In [ ]:
pd_data=pd_data.sort_values(by=['WELL_ID', 'DATE'], ascending=[True, False])
pd_data.reset_index(level=0, inplace=True)

pd_data.head(10)

In [ ]:
df_failure_thingy=pd_data[pd_data['EQUIPMENT_FAILURE'] == 1]
df_failure_thingy.head()

df_failure_thingy=df_failure_thingy[['DATE','WELL_ID']]

df_failure_thingy=df_failure_thingy.rename(index=str, columns={"DATE": "FAILURE_DATE"})

df_failure_thingy

In [ ]:
df_failure_thingy.shape

In [ ]:
df_failure_thingy=df_failure_thingy.drop_duplicates(subset='WELL_ID')
df_failure_thingy.shape

In [ ]:
pd_data=pd_data.sort_values(by=['WELL_ID'], ascending=[True])
df_failure_thingy=df_failure_thingy.sort_values(by=['WELL_ID'], ascending=[True])

In [ ]:
pd_data =pd_data.merge(df_failure_thingy, on=['WELL_ID'], how='left')


In [ ]:
pd_data=pd_data.sort_values(by=['WELL_ID', 'DATE'], ascending=[True, False])

pd_data['FAILURE_DATE'] = pd.to_datetime(pd_data['FAILURE_DATE'])
pd_data['DATE'] = pd.to_datetime(pd_data['DATE'])
pd_data['C'] = pd_data['FAILURE_DATE'] - pd_data['DATE']

pd_data['TIME_TO_FAILURE'] = pd_data['C'] / np.timedelta64(1, 'D')

In [ ]:
pd_data=pd_data.sort_values(by=['WELL_ID', 'DATE'], ascending=[True, False])

In [ ]:
pd.set_option('display.max_rows', 50000)


In [ ]:
pd_data['FAILURE_TARGET'] = np.where(((pd_data.TIME_TO_FAILURE <= 30) & ((pd_data.TIME_TO_FAILURE>=0))), 1, 0)

pd_data.head(10)

In [ ]:
tips_summed = pd_data.groupby(['WELL_ID'])['FAILURE_TARGET'].sum()
tips_summed


In [ ]:
tips_summed = pd_data.groupby(['FAILURE_TARGET'])['S1'].count()
tips_summed

In [ ]:
xxxx=pd_data[pd_data['WELL_ID'] ==100017]

xxxx.head()

In [ ]:
pd_well_id=pd_data.drop_duplicates(subset='WELL_ID')
pd_well_id=pd_well_id[['WELL_ID']]


In [ ]:
pd_well_id.head(5)

In [ ]:
pd_well_id.shape

In [ ]:
pd_well_id.head(100)

In [ ]:
pd_well_id['wookie'] = (np.random.randint(0, 10000, pd_well_id.shape[0]))/10000

In [ ]:
pd_well_id['MODELING_GROUP'] = np.where(((pd_well_id.wookie <= 0.30)), 'VALIDATION', 'MODELING')


In [ ]:
pd_data=pd_data.sort_values(by=['WELL_ID'], ascending=[True])
pd_well_id=pd_well_id.sort_values(by=['WELL_ID'], ascending=[True])

In [ ]:
pd_data =pd_data.merge(pd_well_id, on=['WELL_ID'], how='left')

pd_data.head()

In [ ]:
pd_data=pd_data.drop(columns=['FAILURE_REPAIR_COST', 'FAILURE_PRODUCTION_COST', 'MAINTENANCE_COST',
       'MAINTENANCE_PRODUCTION_LOSS', 'wookie','C','FAILURE_DATE','TIME_TO_FAILURE'])

In [ ]:
pd_data.head()

In [ ]:
df_validation=pd_data[pd_data['MODELING_GROUP'] == 'VALIDATION']


df_validation=df_validation.drop(columns=['MODELING_GROUP'])
df_validation.shape

In [ ]:
df_modeling=pd_data[pd_data['MODELING_GROUP'] == 'MODELING']

df_modeling=df_modeling.drop(columns=['MODELING_GROUP','EQUIPMENT_FAILURE'])
df_modeling.shape

In [ ]:
training_features=df_modeling[['REGION_CLUSTER','MAINTENANCE_VENDOR','MANUFACTURER','WELL_GROUP','TIME_SINCE_MAINTENANCE', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7',
       'S8', 'S9', 'S10', 'S11', 'S12', 'S13', 'S14', 'S15', 'S16', 'S17',
       'S18', 'S19', 'S20', 'S21', 'S22']]
#validation_features=df_validation[['WELL_ID','DATE''REGION_CLUSTER','MAINTENANCE_VENDOR','MANUFACTURER','WELL_GROUP','TIME_SINCE_MAINTENANCE', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7',
   #    'S8', 'S9', 'S10', 'S11', 'S12', 'S13', 'S14', 'S15', 'S16', 'S17',
    #   'S18', 'S19', 'S20', 'S21', 'S22']]

In [ ]:
training_target=df_modeling[['FAILURE_TARGET']]
#validation_target=df_validation[['FAILURE_TARGET']]


In [ ]:
!pip install --upgrade imblearn --no-cache | tail -n 1

In [ ]:
!pip install numpy

In [ ]:
import numpy.dual as dual

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SMOTENC
sm = SMOTE(random_state=12, ratio = 1.0)
smx = SMOTENC(random_state=12,  categorical_features=[0, 1, 2, 3])

In [ ]:
x_res, y_res = smx.fit_sample(training_features, training_target)

In [ ]:
df_x=pd.DataFrame(x_res)

df_x.columns = [
'REGION_CLUSTER','MAINTENANCE_VENDOR','MANUFACTURER','WELL_GROUP','TIME_SINCE_MAINTENANCE', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7',
       'S8', 'S9', 'S10', 'S11', 'S12', 'S13', 'S14', 'S15', 'S16', 'S17',
       'S18', 'S19', 'S20', 'S21', 'S22']
df_x.head()


In [ ]:
df_x.shape

In [ ]:
df_y=pd.DataFrame(y_res)
df_y.columns = ['FAILURE_TARGET']
df_y.head()


In [ ]:
df_y.shape

In [ ]:
df_y.mean(axis = 0) 

In [ ]:
df_modeling = pd.concat([df_y, df_x], axis=1)


In [ ]:
tips_summed = df_modeling.groupby(['REGION_CLUSTER'])['FAILURE_TARGET'].mean()
tips_summed



In [ ]:
df_modeling.head()

In [ ]:
df_modeling['wookie'] = (np.random.randint(0, 10000, df_modeling.shape[0]))/10000
df_modeling=df_modeling[df_modeling['wookie'] <0.40]

In [ ]:
df_modeling=df_modeling.drop(columns=['wookie'])



In [ ]:
df_modeling.mean(axis = 0) 

In [ ]:
df_modelingx = spark.createDataFrame(df_modeling)
df_validationx=spark.createDataFrame(df_validation)
df_modelingx.head()

## Explore data

In [ ]:
df_modelingx.printSchema()

In [ ]:
print("Number of records: " + str(df_modelingx.count()))

# Create a model

In [ ]:
spark_df = df_modelingx
#(train_data, test_data) = spark_df.randomSplit([0.8, 0.2], 24)

train_data=spark_df

MODEL_NAME = "Equipment Failure Model"
DEPLOYMENT_NAME = "Equipment Failure Model"

print("Number of records for training: " + str(train_data.count()))
#print("Number of records for evaluation: " + str(test_data.count()))

spark_df.printSchema()

In [ ]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline, Model
from pyspark.ml import linalg



In [ ]:
si_REGION_CLUSTER = StringIndexer(inputCol = 'REGION_CLUSTER', outputCol = 'REGION_CLUSTER_IX')
si_MAINTENANCE_VENDOR = StringIndexer(inputCol = 'MAINTENANCE_VENDOR', outputCol = 'MAINTENANCE_VENDOR_IX')
si_MANUFACTURER = StringIndexer(inputCol = 'MANUFACTURER', outputCol = 'MANUFACTURER_IX')
si_WELL_GROUP = StringIndexer(inputCol = 'WELL_GROUP', outputCol = 'WELL_GROUP_IX')


In [ ]:
si_Label = StringIndexer(inputCol="FAILURE_TARGET", outputCol="label").fit(train_data)
label_converter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=si_Label.labels)

In [ ]:
va_features = VectorAssembler(inputCols=['REGION_CLUSTER_IX', 'MAINTENANCE_VENDOR_IX',
       'MANUFACTURER_IX', 'WELL_GROUP_IX', 'TIME_SINCE_MAINTENANCE', 'S1', 'S2',
       'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10', 'S11', 'S12', 'S13',
       'S14', 'S15', 'S16', 'S17', 'S18', 'S19', 'S20', 'S21', 'S22'], outputCol="features")

In [ ]:
from pyspark.ml.classification import RandomForestClassifier
classifier = RandomForestClassifier(featuresCol="features")
from pyspark.ml.classification import GBTClassifier
#classifier = GBTClassifier(featuresCol="features")

pipeline = Pipeline(stages=[ si_Label, si_REGION_CLUSTER, si_MAINTENANCE_VENDOR, si_MANUFACTURER, si_WELL_GROUP, va_features, classifier, label_converter])
model = pipeline.fit(train_data)

In [ ]:
predictions = model.transform(train_data)
evaluatorDT = BinaryClassificationEvaluator(rawPredictionCol="prediction")
area_under_curve = evaluatorDT.evaluate(predictions)

#default evaluation is areaUnderROC
print("areaUnderROC = %g" % area_under_curve)

In [ ]:
predictions = model.transform(df_validationx)
evaluatorDT = BinaryClassificationEvaluator(rawPredictionCol="prediction")
area_under_curve = evaluatorDT.evaluate(predictions)

#default evaluation is areaUnderROC
print("areaUnderROC = %g" % area_under_curve)

In [ ]:
predictions.head()

In [ ]:
df_output=predictions.toPandas() 

In [ ]:
df_output.columns

In [ ]:
df_output=df_output.sort_values(by=['WELL_ID', 'DATE'], ascending=[True, False])

In [ ]:
df=df_output
df.head()

In [ ]:
df['FLIPPER'] = np.where(((df.WELL_ID.eq(df.WELL_ID.shift()))), 0,1 )

In [ ]:
df['DINGDONG'] = np.where(((df.FLIPPER==0) & (df.EQUIPMENT_FAILURE==0)), 0,1)

In [ ]:
wookie=df[['index', 'WELL_ID', 'DATE', 'EQUIPMENT_FAILURE', 'FAILURE_TARGET', 'label', 'REGION_CLUSTER_IX',
       'MAINTENANCE_VENDOR_IX', 'MANUFACTURER_IX', 'WELL_GROUP_IX', 'features',
       'rawPrediction', 'probability', 'prediction', 'predictedLabel','FLIPPER','DINGDONG']]



wookie.head(1)

In [ ]:
df['cumsum'] = df['predictedLabel'].cumsum()

In [ ]:
df['bootie']=1
df['dingle']= df.groupby(['cumsum'])['bootie'].cumsum()

In [ ]:
wookie=df[['index', 'WELL_ID', 'DATE', 'EQUIPMENT_FAILURE', 'FAILURE_TARGET', 'label', 'REGION_CLUSTER_IX',
       'MAINTENANCE_VENDOR_IX', 'MANUFACTURER_IX', 'WELL_GROUP_IX', 'features',
       'rawPrediction', 'probability', 'prediction', 'predictedLabel','FLIPPER','DINGDONG','cumsum','bootie','dingle']]



wookie.head(1000)

In [ ]:
dater=df[['DATE']]
dater.shape

In [ ]:
dater=dater.drop_duplicates(subset='DATE')
dater.shape

In [ ]:
dater['DATE_KEY'] = np.arange(len(dater))

In [ ]:
df['FLIPPER'] = np.where(((df.WELL_ID.eq(df.WELL_ID.shift()))), 0,1 )

In [ ]:
dater=dater.sort_values(by=['DATE'], ascending=[True])
#dater.reindex(level=1, inplace=False)
dater['COUNTER']=dater['DATE_KEY']
dater=dater.set_index('COUNTER')
dater.head(1)

In [ ]:
df =df.merge(dater, on=['DATE'], how='left')

In [ ]:
df.head()

In [ ]:
df['FLIPPER'] = np.where(((df.WELL_ID.eq(df.WELL_ID.shift()))), 0,1 )

In [ ]:
df=df[['WELL_ID', 'DATE', 'EQUIPMENT_FAILURE',
       'rawPrediction', 'probability', 'prediction', 'predictedLabel', 'C',
       'DATE_KEY','FLIPPER']]

In [ ]:
df.head(10000)

In [ ]:
df.loc[0, 'C'] = df.loc[0, 'index']

In [ ]:
for i in range(1, len(df)):
    df.loc[i, 'C'] = df.loc[i-1, 'C']+1

In [ ]:
df_output.head()

In [ ]:
df_output['match'] = df_output.WELL_ID.eq(df_output.WELL_ID.shift())

In [ ]:
df_output['FLIPPER'] = np.where(((df_output.WELL_ID.eq(df_output.WELL_ID.shift()))), 0,1 )

In [ ]:
df_output['THECOUNTER'] = np.where((df_output.FLIPPER==1), 0,df_output.THECOUNTER.shift()+1 )

In [ ]:
df_output=df_output.sort_values(by=['WELL_ID'], ascending=[True])
df_output.reset_index(level=1, inplace=True)

df_output.head(10)

In [ ]:
df_failure_actual=df_output[df_output['EQUIPMENT_FAILURE'] == 1]


df_failure_actual=df_failure_actual[['DATE','WELL_ID']]

df_failure_actual=df_failure_actual.rename(index=str, columns={"DATE": "FAILURE_DATE"})
df_failure_actual.head()

In [ ]:

df_failure_actual=df_failure_actual.drop_duplicates(subset='WELL_ID')
df_failure_actual.shape

In [ ]:
df_output =df_output.merge(df_failure_actual, on=['WELL_ID'], how='left')

In [ ]:
df_output.head()

In [ ]:
import datetime as dt     
df_output['TODAY'] =pd.to_datetime('now')

In [ ]:

df_output['P_FAIL_DATE'] = np.where(((df_output.predictedLabel == '1')), df_output.DATE,df_output.TODAY )

In [ ]:
df_output.head()

In [ ]:
df_output=df_output.sort_values(by=['WELL_ID', 'DATE'], ascending=[True, False])

df_output['FAILURE_DATE'] = pd.to_datetime(df_output['FAILURE_DATE'])
df_output['P_FAIL_DATE'] = pd.to_datetime(df_output['P_FAIL_DATE'])
df_output['C'] = df_output['FAILURE_DATE'] - df_output['P_FAIL_DATE']

df_output['WARNING_DAYS'] = df_output['C'] / np.timedelta64(1, 'D')

In [ ]:
df_output['TRUE_POSITIVE_W'] = np.where(((df_output.WARNING_DAYS <= 90) & ((df_output.WARNING_DAYS>=0)) & ((df_output.predictedLabel == '1'))), 1, 0)

In [ ]:
true_positive=df_output[df_output['predictedLabel'] == '1']
true_positive=true_positive[['WELL_ID','WARNING_DAYS','FAILURE_DATE']]

true_positive=true_positive[true_positive['WARNING_DAYS'] >= 0]
true_positive=true_positive[true_positive['WARNING_DAYS'] <=90 ]

true_positive['EQUIPMENT_FAILURE']=1
true_positive['TRUE_POSITIVE']=1

true_positive.head(4)


In [ ]:
true_positive=true_positive.drop_duplicates(subset='WELL_ID', keep='last')
true_positive

In [ ]:
df_output=df_output[['index', 'WELL_ID', 'DATE', 'REGION_CLUSTER', 'MAINTENANCE_VENDOR',
       'MANUFACTURER', 'WELL_GROUP', 'TIME_SINCE_MAINTENANCE', 'S1', 'S2',
       'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10', 'S11', 'S12', 'S13',
       'S14', 'S15', 'S16', 'S17', 'S18', 'S19', 'S20', 'S21', 'S22',
       'EQUIPMENT_FAILURE', 'FAILURE_TARGET', 'label', 'REGION_CLUSTER_IX',
       'MAINTENANCE_VENDOR_IX', 'MANUFACTURER_IX', 'WELL_GROUP_IX', 'features',
       'rawPrediction', 'probability', 'prediction', 'predictedLabel',
       'FAILURE_DATE', 'TODAY', 'P_FAIL_DATE', 'TRUE_POSITIVE_W']]

In [ ]:
df_output=df_output.sort_values(by=['WELL_ID'], ascending=[True])
true_positive=true_positive.sort_values(by=['WELL_ID'], ascending=[True])
df_output =df_output.merge(true_positive, on=['WELL_ID'], how='left')

In [ ]:
df_output['TRUE_POSITIVE'] = np.where(((df_output.WARNING_DAYS <= 90) & ((df_output.WARNING_DAYS>=0)) & ((df_output.predictedLabel == '1'))), 1, 0)

In [ ]:
df_output.head()

In [ ]:
df_output[df_output['EQUIPMENT_FAILURE'] == 1]

In [ ]:
xxx=df_output[df_output['WELL_ID'] == 100001]
pd.set_option('display.max_rows', 50000)

In [ ]:
xxx=xxx[['index', 'WELL_ID', 'DATE', 
       'EQUIPMENT_FAILURE', 'FAILURE_TARGET', 'label', 
       'rawPrediction', 'probability', 'prediction', 'predictedLabel', 'TODAY',
       'P_FAIL_DATE', 'FAILURE_DATE', 'C', 'WARNING_DAYS', 'TRUE_POSITIVE']]

xxx

In [ ]:
tips_summed = df_output.groupby(['label','predictedLabel'])['S1'].count()
tips_summed

In [ ]:
df_confusion = pd.crosstab(df_output['EQUIPMENT_FAILURE'], df_output['prediction'])
df_confusion

In [ ]:
df_output['y']= (df_output['probability'])
df_output['phat'] = df_output.y.str.slice(start=1, stop=2, step=None)
df_output['phat'] = df_output['phat'].astype(float)
df_output=df_output.drop(columns=['y'])
df_output

In [ ]:
z=(pd.qcut(df_output['S1'], 4, labels=False, retbins=True, precision=3, duplicates='raise'))
z=pd.DataFrame(z)
z


In [ ]:
df_output.groupby(pd.qcut(df_output.S1, 10))['predictedLabel'].mean()

# Save and deploy the model

from scipy import sparse
from scipy import linalg

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient 
import json 



wml_client = WatsonMachineLearningAPIClient(WML_CREDENTIALS)

### Remove existing model and deployment

In [ ]:
model_deployment_ids = wml_client.deployments.get_uids()
for deployment_id in model_deployment_ids:
    deployment = wml_client.deployments.get_details(deployment_id)
    model_id = deployment['entity']['deployable_asset']['guid']
    if deployment['entity']['name'] == DEPLOYMENT_NAME:
        print('Deleting deployment id', deployment_id)
        wml_client.deployments.delete(deployment_id)
        print('Deleting model id', model_id)
        wml_client.repository.delete(model_id)
wml_client.repository.list_models()

In [ ]:
model_props = {
    wml_client.repository.ModelMetaNames.NAME: "{}".format(MODEL_NAME),
    wml_client.repository.ModelMetaNames.EVALUATION_METHOD: "binary",
    wml_client.repository.ModelMetaNames.EVALUATION_METRICS: [
        {
           "name": "areaUnderROC",
           "value": area_under_curve,
           "threshold": 0.85
        }
    ]
}

In [ ]:
wml_models = wml_client.repository.get_details()
model_uid = None
for model_in in wml_models['models']['resources']:
    if MODEL_NAME == model_in['entity']['name']:
        model_uid = model_in['metadata']['guid']
        break

if model_uid is None:
    print("Storing model ...")

    published_model_details = wml_client.repository.store_model(model=model, meta_props=model_props, training_data=train_data, pipeline=pipeline)
    model_uid = wml_client.repository.get_model_uid(published_model_details)
    print("Done")

In [ ]:
model_uid

In [ ]:
wml_deployments = wml_client.deployments.get_details()
deployment_uid = None
for deployment in wml_deployments['resources']:
    if DEPLOYMENT_NAME == deployment['entity']['name']:
        deployment_uid = deployment['metadata']['guid']
        break

if deployment_uid is None:
    print("Deploying model...")

    deployment = wml_client.deployments.create(artifact_uid=model_uid, name=DEPLOYMENT_NAME, asynchronous=False)
    deployment_uid = wml_client.deployments.get_uid(deployment)
    
print("Model id: {}".format(model_uid))
print("Deployment id: {}".format(deployment_uid))

# Configure OpenScale

In [ ]:
from ibm_ai_openscale import APIClient
from ibm_ai_openscale.engines import *
from ibm_ai_openscale.utils import *
from ibm_ai_openscale.supporting_classes import PayloadRecord, Feature
from ibm_ai_openscale.supporting_classes.enums import *

### Get AI OpenScale GUID

In [ ]:
import requests

AIOS_GUID = None
token_data = {
    'grant_type': 'urn:ibm:params:oauth:grant-type:apikey',
    'response_type': 'cloud_iam',
    'apikey': CLOUD_API_KEY
}

response = requests.post('https://iam.bluemix.net/identity/token', data=token_data)
iam_token = response.json()['access_token']
iam_headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer %s' % iam_token
}

resources = json.loads(requests.get('https://resource-controller.cloud.ibm.com/v2/resource_instances', headers=iam_headers).text)['resources']
for resource in resources:
    if "aiopenscale" in resource['id'].lower():
        AIOS_GUID = resource['guid']
        
AIOS_CREDENTIALS = {
    "instance_guid": AIOS_GUID,
    "apikey": CLOUD_API_KEY,
    "url": "https://api.aiopenscale.cloud.ibm.com"
}

if AIOS_GUID is None:
    print('AI OpenScale GUID NOT FOUND')
else:
    print(AIOS_GUID)

## Create schema and datamart

In [ ]:
ai_client = APIClient(aios_credentials=AIOS_CREDENTIALS)
ai_client.version
time.sleep(20)

### Set up datamart

In [ ]:
try:
    data_mart_details = ai_client.data_mart.get_details()
    if 'internal_database' in data_mart_details and data_mart_details['internal_database']:
        if KEEP_MY_INTERNAL_POSTGRES:
            print('Using existing internal datamart.')
        else:
            if DB_CREDENTIALS is None:
                print('No postgres credentials supplied. Using existing internal datamart')
            else:
                print('Switching to external datamart')
                ai_client.data_mart.delete(force=True)
                ai_client.data_mart.setup(db_credentials=DB_CREDENTIALS)
    else:
        print('Using existing external datamart')
except:
    if DB_CREDENTIALS is None:
        print('Setting up internal datamart')
        ai_client.data_mart.setup(internal_db=True)
    else:
        print('Setting up external datamart')
        ai_client.data_mart.setup(db_credentials=DB_CREDENTIALS)
    

In [ ]:
data_mart_details = ai_client.data_mart.get_details()
data_mart_details

## Bind machine learning engines

In [ ]:
binding_uid = ai_client.data_mart.bindings.add('WML instance', WatsonMachineLearningInstance(WML_CREDENTIALS))
if binding_uid is None:
    binding_uid = ai_client.data_mart.bindings.get_details()['service_bindings'][0]['metadata']['guid']
bindings_details = ai_client.data_mart.bindings.get_details()
ai_client.data_mart.bindings.list()

In [ ]:
print(binding_uid)

In [ ]:
ai_client.data_mart.bindings.list_assets()

## Subscriptions

### Remove existing  subscriptions

In [ ]:
subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
for subscription in subscriptions_uids:
    sub_name = ai_client.data_mart.subscriptions.get_details(subscription)['entity']['asset']['name']
    if sub_name == MODEL_NAME:
        ai_client.data_mart.subscriptions.delete(subscription)
        print('Deleted existing subscription for', MODEL_NAME)

In [ ]:
subscription = ai_client.data_mart.subscriptions.add(WatsonMachineLearningAsset(
    model_uid,
    problem_type=ProblemType.BINARY_CLASSIFICATION,
    input_data_type=InputDataType.STRUCTURED,
    label_column='BAD_YIELD',
    prediction_column='predictedLabel',
    probability_column='probability',
    feature_columns = ['WS_001_FLOW_MEAN', 'WS_001_FLOW_MIN', 'WS_001_FLOW_MAX',
       'WS_001_CONC_MEAN', 'WS_001_CONC_MIN', 'WS_001_CONC_MAX',
       'DMW_FLOW_MEAN', 'DMW_FLOW_MIN', 'DMW_FLOW_MAX', 'ALK_FLOW_MEAN',
       'ALK_FLOW_MIN', 'ALK_FLOW_MAX', 'RPM_MEAN', 'RPM_MIN', 'RPM_MAX','PLANT_A'],
    categorical_columns = ['PLANT_A']
))

if subscription is None:
    print('Subscription already exists; get the existing one')
    subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
    for sub in subscriptions_uids:
        if ai_client.data_mart.subscriptions.get_details(sub)['entity']['asset']['name'] == MODEL_NAME:
            subscription = ai_client.data_mart.subscriptions.get(sub)

Get subscription list

In [ ]:
subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
ai_client.data_mart.subscriptions.list()

In [ ]:

subscription.get_details()

### Score the model so we can configure monitors

In [ ]:
propensity_to_buy_scoring_endpoint = None
print(deployment_uid)

for deployment in wml_client.deployments.get_details()['resources']:
    if deployment_uid in deployment['metadata']['guid']:
        propensity_to_buy_scoring_endpoint = deployment['entity']['scoring_url']
        
print(propensity_to_buy_scoring_endpoint)

In [ ]:
fields = ['WS_001_FLOW_MEAN', 'WS_001_FLOW_MIN', 'WS_001_FLOW_MAX',
       'WS_001_CONC_MEAN', 'WS_001_CONC_MIN', 'WS_001_CONC_MAX',
       'DMW_FLOW_MEAN', 'DMW_FLOW_MIN', 'DMW_FLOW_MAX', 'ALK_FLOW_MEAN',
       'ALK_FLOW_MIN', 'ALK_FLOW_MAX', 'RPM_MEAN', 'RPM_MIN', 'RPM_MAX', 'PLANT_A']
values = [[91.472265,91.591823,91.400894,29.999306,29.9453,30.0347,88.597187,88.598325,88.595852,91.467674,91.52794,91.439629,5010.304375,5010.05,5010.61,1],
          [91.286568,91.333725,91.225539,29.999756,29.8993,30.1047,88.597187,88.598325,88.595852,91.467674,91.52794,91.439629,5011.043681,5010.74,5011.32,1],
          [91.581607,91.677855,91.57625,30.000624,29.9481,30.0648,88.597187,88.598325,88.595852,91.467674,91.52794,91.439629,5008.680625,5007.45,5009.42,0],
          [91.332992,91.419757,91.313216,30.000023,29.9221,30.1148,88.597187,88.598325,88.595852,91.467674,91.52794,91.439629,5010.7575,5010.47,5011.09,1],
          [87.178007,87.376229,87.19236,65.001133,64.8447,65.1378,92.93055,92.934958,92.91834,87.957784,88.105276,87.929657,5009.978056,5009.61,5010.3,1],
          [91.463103,91.591823,91.400894,29.999842,29.9367,30.0449,88.597187,88.598325,88.595852,91.467674,91.52794,91.439629,5010.263889,5009.75,5010.5,0],
          [87.284294,87.462261,87.280037,65.000094,64.9171,65.0708,92.93055,92.934958,92.91834,87.957784,88.105276,87.929657,5008.015347,5006.6,5009.73,1],
          [87.121198,87.290196,87.104682,65.000774,64.8889,65.1762,92.93055,92.934958,92.91834,86.453545,86.63842,86.425383,5009.210625,5008.67,5009.67,1],
          [87.2452,87.462261,87.19236,62.999506,62.8072,63.167,92.682929,92.68715,92.67134,87.957784,88.105276,87.929657,5011.408889,5011.12,5011.59,0],
          [87.099207,87.032099,87.280037,64.983894,62.8779,65.1535,92.93055,92.934958,92.91834,86.453545,86.63842,86.425383,5010.101181,5009.21,5010.83,0],
          [91.243197,91.333725,91.225539,29.998703,29.9305,30.1019,88.597187,88.598325,88.595852,91.467674,91.52794,91.439629,5010.714444,5010.44,5011.11,1],
          [91.612149,91.677855,91.57625,29.998469,29.9674,30.0269,88.597187,88.598325,88.595852,91.467674,91.52794,91.439629,5009.607431,5008.38,5010.55,1],
          [87.005748,87.204164,86.929326,65.001815,64.1377,65.9022,92.93055,92.934958,92.91834,86.453545,86.63842,86.425383,5010.824861,5010.53,5011.05,1],
          [91.260301,91.333725,91.225539,30.000992,29.9268,30.0831,88.597187,88.598325,88.595852,91.467674,91.52794,91.439629,5010.054444,5009.72,5010.31,1],
          [90.70443,90.731497,90.699472,29.513889,29.0,30.0,88.597187,88.598325,88.595852,90.464848,90.550036,90.43678,5007.784861,5006.61,5008.85,1],
          [91.538237,91.677855,91.488572,30.000409,29.9454,30.0518,88.597187,88.598325,88.595852,91.467674,91.52794,91.439629,5009.791319,5008.99,5010.11,1],
          [87.051561,87.204164,87.017004,64.999602,64.8108,65.146,92.93055,92.934958,92.91834,86.453545,86.63842,86.425383,5009.726389,5005.68,5010.44,0],
          [91.614593,91.763888,91.57625,29.999788,29.9509,30.0482,88.597187,88.598325,88.595852,91.467674,91.52794,91.439629,5008.214444,5006.21,5009.69,0],
          [91.365367,91.419757,91.313216,30.00108,29.8762,30.1167,88.597187,88.598325,88.595852,91.467674,91.52794,91.439629,4998.986875,4996.14,5003.11,0]
         ]
payload_scoring = {"fields": fields,"values": values}
scoring_response = wml_client.deployments.score(propensity_to_buy_scoring_endpoint, payload_scoring)

print(scoring_response)

## Quality and feedback monitoring

### Enable quality monitoring

Wait ten seconds to allow the payload logging table to be set up before we begin enabling monitors.

In [ ]:
time.sleep(20)
subscription.quality_monitoring.enable(threshold=0.7, min_records=100)

### Feedback logging

In [ ]:
!rm df_feedback.json
!wget https://raw.githubusercontent.com/shadgriffin/oglabworking/master/df_feedback.json

In [ ]:
with open('df_feedback.json') as feedback_file:
    df_feedback = json.load(feedback_file)
subscription.feedback_logging.store(df_feedback)

In [ ]:
subscription.feedback_logging.show_table()

In [ ]:
run_details = subscription.quality_monitoring.run()
status = run_details['status']
id = run_details['id']
print(id)

print("Run status: {}".format(status))

start_time = time.time()
elapsed_time = 0

while status != 'completed' and elapsed_time < 60:
    time.sleep(10)
    run_details = subscription.quality_monitoring.get_run_details(run_uid=id)
    status = run_details['status']
    elapsed_time = time.time() - start_time
    print("Run status: {}".format(status))

In [ ]:
subscription.quality_monitoring.get_run_details()

In [ ]:
subscription.quality_monitoring.show_table()

subscription.quality_monitoring._get_data_from_rest_api()

In [ ]:
ai_client.data_mart.get_deployment_metrics()

## Fairness monitoring

In [ ]:
subscription.fairness_monitoring.enable(
            features=[
                Feature("PLANT_A", majority=[[1,1]], minority=[[0,0]], threshold=0.95)
            ],
            favourable_classes=['GOOD'],
            unfavourable_classes=['BAD'],
            min_records=1000,
            training_data=df_training
        )

## Score the model again now that monitoring is configured

In [ ]:
!rm df_payload_biased.json
!wget https://raw.githubusercontent.com/shadgriffin/oglabworking/master/df_payload_biased.json

Score 1000 randomly chosen records

In [ ]:
import random

with open('df_payload_biased.json', 'r') as scoring_file:
    scoring_data = json.load(scoring_file)

fields = scoring_data['fields']
values = []
for _ in range(1000):
    values.append(random.choice(scoring_data['values']))
payload_scoring = {"fields": fields, "values": values}

scoring_response = wml_client.deployments.score(propensity_to_buy_scoring_endpoint, payload_scoring)
print(scoring_response)

In [ ]:
subscription.get_details()

# Insert historical payloads

In [ ]:
!rm payload_history*.json
!wget https://raw.githubusercontent.com/shadgriffin/propensitytobuylab/master/payload_history_1.json
!wget https://raw.githubusercontent.com/shadgriffin/propensitytobuylab/master/payload_history_2.json
!wget https://raw.githubusercontent.com/shadgriffin/propensitytobuylab/master/payload_history_3.json
!wget https://raw.githubusercontent.com/shadgriffin/propensitytobuylab/master/payload_history_4.json
!wget https://raw.githubusercontent.com/shadgriffin/propensitytobuylab/master/payload_history_5.json
!wget https://raw.githubusercontent.com/shadgriffin/propensitytobuylab/master/payload_history_6.json
!wget https://raw.githubusercontent.com/shadgriffin/propensitytobuylab/master/payload_history_7.json

In [ ]:
historyDays = 7
from ibm_ai_openscale.supporting_classes import PayloadRecord, Feature
import datetime
import time

In [ ]:
data_mart_id = subscription.get_details()['metadata']['url'].split('/service_bindings')[0].split('marts/')[1]
print(data_mart_id)

In [ ]:
performance_metrics_url = 'https://api.aiopenscale.cloud.ibm.com' + subscription.get_details()['metadata']['url'].split('/service_bindings')[0] + '/metrics'
print(performance_metrics_url)

## Insert historical fairness metrics

In [ ]:
!rm fairness_records.json
!wget https://raw.githubusercontent.com/shadgriffin/oglabworking/master/fairness_records.json
import random

In [ ]:

token_data = {
    'grant_type': 'urn:ibm:params:oauth:grant-type:apikey',
    'response_type': 'cloud_iam',
    'apikey': AIOS_CREDENTIALS['apikey']
}

response = requests.post('https://iam.bluemix.net/identity/token', data=token_data)
iam_token = response.json()['access_token']
iam_headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer %s' % iam_token
}

with open('fairness_records.json', 'r') as history_file:
    payloads = json.load(history_file)

for day in range(historyDays):
    print('Day', day + 1)
    for hour in range(24):
        score_time = (datetime.datetime.utcnow() + datetime.timedelta(hours=(-(24*day + hour + 1)))).strftime('%Y-%m-%dT%H:%M:%SZ')
        
        qualityMetric = {
            'metric_type': 'fairness',
            'binding_id': binding_uid,
            'timestamp': score_time,
            'subscription_id': model_uid,
            'asset_revision': model_uid,
            'deployment_id': deployment_uid,
            'value': random.choice(payloads)
        }

        response = requests.post(performance_metrics_url, json=[qualityMetric], headers=iam_headers)
print('Finished')

## Insert historical quality metrics

In [ ]:
token_data = {
    'grant_type': 'urn:ibm:params:oauth:grant-type:apikey',
    'response_type': 'cloud_iam',
    'apikey': AIOS_CREDENTIALS['apikey']
}

response = requests.post('https://iam.bluemix.net/identity/token', data=token_data)
iam_token = response.json()['access_token']
iam_headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer %s' % iam_token
}

measurements = [0.94, 0.91, 0.78, 0.82, 0.90, 0.94, 0.93]
for day in range(historyDays):
    print('Day', day + 1)
    for hour in range(24):
        score_time = (datetime.datetime.utcnow() + datetime.timedelta(hours=(-(24*day + hour + 1)))).strftime('%Y-%m-%dT%H:%M:%SZ')
        
        qualityMetric = {
            'metric_type': 'quality',
            'binding_id': binding_uid,
            'timestamp': score_time,
            'subscription_id': model_uid,
            'asset_revision': model_uid,
            'deployment_id': deployment_uid,
            'value': {
                'quality': measurements[day],
                'threshold': 0.85,
                'metrics': [
                    {
                        'name': 'auroc',
                        'value': measurements[day],
                        'threshold': 0.75
                    }
                ]
            }
        }

        response = requests.post(performance_metrics_url, json=[qualityMetric], headers=iam_headers)
print('Finished')

## Insert historical performance metrics

In [ ]:
token_data = {
    'grant_type': 'urn:ibm:params:oauth:grant-type:apikey',
    'response_type': 'cloud_iam',
    'apikey': AIOS_CREDENTIALS['apikey']
}

response = requests.post('https://iam.bluemix.net/identity/token', data=token_data)
iam_token = response.json()['access_token']
iam_headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer %s' % iam_token
}

for day in range(historyDays):
    print('Day', day + 1)
    for hour in range(24):
        score_time = (datetime.datetime.utcnow() + datetime.timedelta(hours=(-(24*day + hour + 1)))).strftime('%Y-%m-%dT%H:%M:%SZ')
        score_count = random.randint(600, 6000)
        score_resp = random.uniform(600, 3000)

        performanceMetric = {
            'metric_type': 'performance',
            'binding_id': binding_uid,
            'timestamp': score_time,
            'subscription_id': model_uid,
            'asset_revision': model_uid,
            'deployment_id': deployment_uid,
            'value': {
                'response_time': score_resp,
                'records': score_count
            }
        }

        response = requests.post(performance_metrics_url, json=[performanceMetric], headers=iam_headers)
print('Finished')

## Configure Explainability

In [ ]:
pd_data.head()

In [ ]:
from ibm_ai_openscale.supporting_classes import *
subscription.explainability.enable(training_data=df_training)

In [ ]:
subscription.explainability.get_details()

## Run fairness monitor

Kick off a fairness monitor run on current data. Depending on how fast the monitor runs, the table may not contain the most recent results.

In [ ]:
run_details = subscription.fairness_monitoring.run()

In [ ]:
subscription.fairness_monitoring.show_table()

## Additional data to help debugging

In [ ]:
#print('Datamart:', data_mart_id)
print('Model:', model_uid)
print('Deployment:', deployment_uid)
print('Binding:', binding_uid)
print('Scoring URL:', propensity_to_buy_scoring_endpoint)

## Identify transactions for Explainability

Transaction IDs identified by the cells below can be copied and pasted into the Explainability tab of the OpenScale dashboard.

In [ ]:
import json, random

DEPLOYMENT_NAME = "Yield Model"
MIN_RECORDS = 1000
MAX_RECORDS = 1000

In [ ]:
!rm df_payload_biased.json
!wget https://raw.githubusercontent.com/shadgriffin/oglabworking/master/df_payload_biased.json

In [ ]:
wml_deployments = wml_client.deployments.get_details()
scoring_url = None
for deployment in wml_deployments['resources']:
    if DEPLOYMENT_NAME == deployment['entity']['name']:
        scoring_url = deployment['entity']['scoring_url']
        break
    
print("Scoring URL: {}".format(scoring_url))

In [ ]:
try:
    with open('df_payload_biased.json', 'r') as scoring_file:
        scoring_data = json.load(scoring_file)
    print('file found')
    
except:
    !wget https://raw.githubusercontent.com/shadgriffin/oglabworking/master/df_payload_biased.json
    with open('df_payload_biased.json', 'r') as scoring_file:
        scoring_data = json.load(scoring_file)
    print('file downloaded')


In [ ]:
fields = scoring_data['fields']
values = []
for _ in range(0, random.randint(MIN_RECORDS, MAX_RECORDS)):
    values.append(random.choice(scoring_data['values']))
payload_scoring = {"fields": fields, "values": values}

scoring_response = wml_client.deployments.score(scoring_url, payload_scoring)
print(scoring_response)

In [ ]:
time.sleep(10)
payload_data = subscription.payload_logging.get_table_content(limit=200)
payload_data.filter(items=['scoring_id', 'predictedLabel', 'probability','PLANT_A'])

## Congratulations!

You have finished the hands-on lab for IBM Watson OpenScale. You can now view the [OpenScale Dashboard](https://aiopenscale.cloud.ibm.com/). Click on the tile for the Propensity to Buy model to see fairness, accuracy, and performance monitors. Click on the timeseries graph to get detailed information on transactions during a specific time window.

